# Plot lineage tree

In [1]:
import pandas as pd

# Load your CSV
df = pd.read_csv("output/protein-evolution-database_V4_proteins_reactions_clean.csv")  # change filename accordingly


In [4]:
enzyme_fam = {
    "https://doi.org/10.1038/s41557-021-00794-z": "p450",
    "https://doi.org/10.1021/jacs.9b02931": "p450",
    "doi.org/10.1021/jacs.0c01313": "p450",
    "doi.org/10.1021/acscatal.0c01349": "p450",
    "doi.org/10.1002/anie.202002861": "p450",
    "doi.org/10.1021/jacs.1c11340": "p450",
    "doi.org/10.1038/s41557-019-0343-5": "p450",
    "doi.org/10.1021/acscatal.3c05370": "p450",
    "doi.org/10.1021/jacs.9b04344": "p450",
    "doi.org/10.1021/jacs.3c04870": "p450",
    "doi.org/10.1021/jacs.2c08285": "p450",
    "doi.org/10.1021/acscentsci.3c00516": "p450",
    "doi.org/10.1021/jacs.2c00251": "p450",
    "doi.org/10.1038/s41929-024-01149-w": "p450",
    "doi.org/10.1038/s44160-023-00431-2": "p450",
    "doi.org/10.1002/anie.202303879": "p450",
    "doi.org/10.1021/jacs.0c03428": "p450",
    "doi.org/10.1038/s41929-022-00908-x": "p450",
    "doi.org/10.1021/jacs.3c11722": "p450",
    "doi.org/10.1002/anie.202110873": "p450",
    "DOI: 10.1126/science.adi5554": "p450",
    "https://doi.org/10.1021/jacs.3c08053": "pgb",
    "https://doi.org/10.1002/anie.202208936": "pgb",
    "doi/10.1021/jacs.4c04190": "pgb",
    "doi.org/10.1021/jacs.2c02723": "pgb",
    "doi.org/10.1021/jacs.4c09989": "pgb",
    "DOI: 10.1126/science.aah6219": "rma",
    "doi:10.1038/nature24996": "rma",
    "doi: 10.1055/s-0037-1611662": "rma",
    "doi.org/10.1021/acscatal.0c01888": "rma",
    "doi.org/10.1038/s41589-024-01619-z": "trpb",
    "doi.org/10.1002/anie.202106938": "trpb",
    "doi.org/10.1002/cbic.201900497": "trpb",
    "doi.org/10.1021/jacs.9b09864": "trpb",
    "doi.org/10.1021/acscatal.9b02089": "trpb",
    "https://doi.org/10.1021/jacs.9b11608": "non-heme-iron"
}

In [5]:
# map DOI to enzyme family
df["enzyme_family"] = df["doi"].map(enzyme_fam)

In [22]:
df["id"].nunique()

1341

In [23]:
len(df)

1341

In [6]:
df.columns

Index(['culture_collection_entry', 'enzyme_name_from_paper',
       'Uniprot_ID(if applicable)', 'comment', 'reaction_smiles',
       'parent_DNA_sequence', 'parent_aminoacid_sequence',
       'aminoacid_mutations_from_parent', 'variant_DNA_sequence',
       'mutations_from_parent', 'cofactor', 'additive (if applicable)',
       'additive_CAS', 'enzyme_form', 'substrate_concentration',
       'activity_for_reaction_% (if applicable)', 'TTN (if applicable)',
       'selectivity(ee%),diastereo or chemo should be a separate smiles entry',
       'alternative_product_SMILES', 'failed_substrates (if available)',
       'date published ', 'first author', 'paper title', 'doi', 'SUBMITTED BY',
       'raw data name', 'cannonical_reactions', 'named_reactions', 'errors',
       'parent_aa', 'variant_aa', 'warnings', 'id', 'enzyme_family'],
      dtype='object')

In [18]:
# create a folder called "lineage" in this dircotry to store all the fasta files
import os
if not os.path.exists("lineage"):
    os.makedirs("lineage")
# make a subfolder called "lineage/fasta"
if not os.path.exists("lineage/fasta"):
    os.makedirs("lineage/fasta")
# make a subfolder called "lineage/alinged"
if not os.path.exists("lineage/aligned"):
    os.makedirs("lineage/aligned")
# make a subfolder called "lineage/phylogenetic"
if not os.path.exists("lineage/phylogenetic"):
    os.makedirs("lineage/phylogenetic")

In [26]:
from pathlib import Path

# Track which FASTA files we've already initialized
initialized_families = set()

for _, row in df.iterrows():
    enzyme_family = row["enzyme_family"]
    if pd.isna(enzyme_family):
        continue  # skip if enzyme family is not mapped

    fasta_filename = Path(f"lineage/fasta/{enzyme_family}.fasta")

    # If first time encountering this enzyme family, delete existing file
    if enzyme_family not in initialized_families:
        if fasta_filename.exists():
            fasta_filename.unlink()  # delete file
        initialized_families.add(enzyme_family)

    # Append new entry
    with open(fasta_filename, "a") as fasta_file:
        fasta_file.write(f">{row['id']}\n{row['variant_aa']}\n")


In [27]:
from pathlib import Path
from Bio.Align.Applications import MafftCommandline

# for each fasta in the lineage/fasta directory align the sequences
# and save the aligned sequences in the lineage/aligned folder with the same name

# loop through all the fasta files 
folder = Path("lineage/fasta")

# Find all .fasta files (case-sensitive)
fasta_files = list(folder.glob("*.fasta"))

for fasta_file in fasta_files:
    enzyme_family = fasta_file.stem  # get the enzyme family from the filename
    aligned_filename = f"lineage/aligned/{enzyme_family}_aligned.fasta"

    # run MAFFT to align the sequences
    mafft_cline = MafftCommandline(input=str(fasta_file))
    stdout, stderr = mafft_cline()

    # write the aligned sequences to the corresponding aligned fasta file
    with open(aligned_filename, "w") as aligned_file:
        aligned_file.write(stdout)


In [32]:
from tqdm import tqdm
from pathlib import Path
from Bio import AlignIO, Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import matplotlib.pyplot as plt

# for each aligned fasta in the lineage/aligned directory, create a phylogenetic tree
aligned_folder = Path("lineage/aligned")
phylo_dir = Path("lineage/phylogenetic")
phylo_dir.mkdir(parents=True, exist_ok=True)

aligned_files = list(aligned_folder.glob("*.fasta"))

for aligned_file in tqdm(aligned_files):
    enzyme_family = aligned_file.stem
    phylo_filename = phylo_dir / f"{enzyme_family}.nwk"
    img_filename = phylo_dir / f"{enzyme_family}.png"

    # read the aligned sequences
    alignment = AlignIO.read(str(aligned_file), "fasta")

    # calculate distance matrix
    calculator = DistanceCalculator('identity')
    distance_matrix = calculator.get_distance(alignment)

    # construct the phylogenetic tree
    constructor = DistanceTreeConstructor()
    tree = constructor.nj(distance_matrix)

    # write the tree to a Newick file
    Phylo.write(tree, phylo_filename, "newick")

    # Remove internal node names
    tip_names = set(record.id for record in alignment)
    for clade in tree.get_nonterminals():
        if clade.name not in tip_names:
            clade.name = None

    # Draw the tree to a specific matplotlib figure
    fig = plt.figure(figsize=(10, max(4, len(alignment) * 0.3)))  # auto-adjust height
    ax = fig.add_subplot(1, 1, 1)
    Phylo.draw(tree, do_show=False, axes=ax, branch_labels=None)
    fig.savefig(img_filename, dpi=300, bbox_inches="tight")
    plt.close(fig)


100%|██████████| 5/5 [05:22<00:00, 64.44s/it] 
